In [ ]:
!pip install transformers

     |████████████████████████████████| 7.8MB 1.8MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 97.7MB/s eta 0:00:01
     |████████████████████████████████| 778kB 97.0MB/s eta 0:00:01
     |████████████████████████████████| 1.4MB 121.2MB/s eta 0:00:01
     |████████████████████████████████| 61kB 89.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 106.9MB/s eta 0:00:01


In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
